In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate
from surprise import SVD, SVDpp
from sklearn.preprocessing import MinMaxScaler

In [2]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [83]:
colNames = ['user_id','criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'number']
data_full = pd.read_csv("E:/Research Project/data_movies.txt", sep ="\t",header= None , names= colNames )
data_full.head()

,user_id,criterion1,criterion2,criterion3,criterion4,overall,movie_id,number
0,1,6,6,8,12,8,2,1
1,1,9,11,10,9,10,26,2
2,1,6,10,9,8,7,61,3
3,1,6,6,6,5,5,86,4
4,1,10,11,10,9,10,132,5


In [84]:
data_full.shape

(62156, 8)

In [5]:
print( "The data has ", len(data_full.movie_id.unique()), " uniques id and", len(data_full.user_id.unique()),"unique users")

The data has  976  uniques id and 6078 unique users


In [85]:
X = data_full[['user_id','criterion1', 'criterion2', 'criterion3', 'criterion4', 'movie_id']]
y = data_full['overall']
from sklearn.model_selection import train_test_split
full_train, full_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(full_test.shape, full_train.shape)

(20512, 6) (41644, 6)


In [7]:
data_full.shape

(62156, 8)

In [26]:
# creting a list of individual criteria values
list_of_criteria_train = {}
list_of_criteria_test = {}
dfs = ('df1', 'df2', 'df3', 'df4')
criterion = ('criterion1', 'criterion2', 'criterion3', 'criterion4')
for df, ctn in zip(dfs, criterion):
    print(df, ctn)
    full_dataset_train = full_train[['user_id',"movie_id",ctn]]
    list_of_criteria_train[df] = full_dataset_train
    full_dataset_test = full_test[['user_id',"movie_id",ctn]]
    list_of_criteria_test[df] = full_dataset_test

df1 criterion1
df2 criterion2
df3 criterion3
df4 criterion4


In [31]:
print("train",list_of_criteria_train['df1'].shape, list_of_criteria_train['df2'].shape,
      list_of_criteria_train['df4'].shape, list_of_criteria_train['df3'].shape)
print("test",list_of_criteria_test['df1'].shape, list_of_criteria_test['df2'].shape,
      list_of_criteria_test['df4'].shape, list_of_criteria_test['df3'].shape)

train (41644, 3) (41644, 3) (41644, 3) (41644, 3)
test (20512, 3) (20512, 3) (20512, 3) (20512, 3)


## Individual criteria SVD predictions

In [58]:
def compute_svd_predictions(criteria_data_train, criteria_data_test):
    reader = Reader()
#dataset creation
    criteria_data_train.columns =["userId", "movieId", "rating"]
    criteria_data_test.columns =["userId", "movieId", "rating"]
    
    # scaling to a scale of 5 
    min_max_scaler = MinMaxScaler()
    
    rate = criteria_data_train[['rating']].values.astype(float)
    rate_scaled = min_max_scaler.fit_transform(rate)
    criteria_data_train[['rating']]=rate_scaled*5
    
    rate = criteria_data_test[['rating']].values.astype(float)
    rate_scaled = min_max_scaler.fit_transform(rate)
    criteria_data_test[['rating']]=rate_scaled*5
    
    reader = Reader()
    svd = SVD()
        
    train_data = Dataset.load_from_df(criteria_data_train, reader)
    fullTrainSet = train_data.build_full_trainset()
    
    test_data = Dataset.load_from_df(criteria_data_test, reader)
    training_data, testing_data = train_test_split(test_data, test_size=1.0)
    
    #     print(type(training_data))
    svd.fit(fullTrainSet)
    pred = svd.test(testing_data)
    pdf = pd.DataFrame(pred)
#     pred_data = pdf[['uid',"iid","est"]]
#     pd.merge(predicted_data, pred_data)
    
    return pdf

In [63]:
predicted_dataset = list_of_criteria_test['df1'][["userId","movieId"]]
# new_dataset = new_dataset.set_index(['userId','movieId'])
print(predicted_dataset.shape)
predicted_dataset.head()

(20512, 2)


,userId,movieId
33907,3277,832
25858,2512,13
61694,6036,899
1554,146,722
11072,1093,968


##  Creating new dataset based on ealuated multiple criteria

In [55]:
# new_full_data = full_train
# new_full_data=new_full_data[["user_id", "movie_id", "criterion1", "criterion2", "criterion3", "criterion4"]]
# new_full_data.columns =["userId", "movieId", "df1", "df2", "df3", "df4"]
# new_full_data.head()

In [56]:
# new_full_data.shape

In [65]:
from surprise.model_selection import train_test_split
import random
random.seed(108)
for key in list_of_criteria_train.keys():
    print(key)
    pred_data = compute_svd_predictions(list_of_criteria_train[key],list_of_criteria_test[key])
    pred_data.columns =["userId", "movieId", "Orating", key,"details"]
    new_data = pred_data.drop(columns=["Orating","details"])
    new_data = new_data.set_index(['userId','movieId'])
    print(new_data.shape)
#     print(new_data.head())
    predicted_dataset = new_data.combine_first(predicted_dataset.set_index(['userId','movieId'])).reset_index()
print(predicted_dataset.shape)

df1
(20512, 1)
df2
(20512, 1)
df3
(20512, 1)
df4
(20512, 1)
(20512, 6)


In [66]:
predicted_dataset.head()

,userId,movieId,df1,df2,df3,df4
0,1,26,4.169332,4.050744,3.828402,3.599499
1,1,86,2.716256,3.596563,2.796181,3.272879
2,1,132,3.602458,3.555491,2.993421,3.266680
3,1,180,3.661907,4.010367,4.188141,4.423180
4,1,191,3.897889,3.537554,3.450182,3.420344


In [67]:
full_test.head()

,user_id,criterion1,criterion2,criterion3,criterion4,movie_id
33907,3277,13,12,10,9,832
25858,2512,6,7,7,7,13
61694,6036,13,13,13,13,899
1554,146,7,5,2,2,722
11072,1093,9,9,10,9,968


In [97]:
# normalise the ratings of the original testing and training data columns on 1-5 scale
min_max_scaler = MinMaxScaler()

full_test[['criterion1']] = min_max_scaler.fit_transform(full_test[['criterion1']].values.astype(float))*5
full_test[['criterion2']] = min_max_scaler.fit_transform(full_test[['criterion2']].values.astype(float))*5
full_test[['criterion3']] = min_max_scaler.fit_transform(full_test[['criterion3']].values.astype(float))*5
full_test[['criterion4']] = min_max_scaler.fit_transform(full_test[['criterion4']].values.astype(float))*5

full_train[['criterion1']] = min_max_scaler.fit_transform(full_train[['criterion1']].values.astype(float))*5
full_train[['criterion2']] = min_max_scaler.fit_transform(full_train[['criterion2']].values.astype(float))*5
full_train[['criterion3']] = min_max_scaler.fit_transform(full_train[['criterion3']].values.astype(float))*5
full_train[['criterion4']] = min_max_scaler.fit_transform(full_train[['criterion4']].values.astype(float))*5

In [89]:
y_train = min_max_scaler.fit_transform(pd.DataFrame(y_train).values.astype(float))*5
y_test = min_max_scaler.fit_transform(pd.DataFrame(y_test).values.astype(float))*5


In [95]:
print(pd.DataFrame(y_train).shape, pd.DataFrame(y_test).shape)

(41644, 1) (20512, 1)


In [99]:
full_train.head()

,user_id,criterion1,criterion2,criterion3,criterion4,movie_id
45737,4461,4.166667,3.750000,3.750000,4.166667,574
12903,1263,0.833333,2.500000,1.250000,2.083333,773
55214,5395,5.000000,4.166667,4.166667,5.000000,1
41336,4018,1.250000,2.500000,1.666667,3.750000,612
57367,5593,5.000000,5.000000,5.000000,5.000000,813


## Aggregation Function 

In [114]:
training_data = full_train[['criterion1','criterion2','criterion3','criterion4']]
print(training_data.shape)
training_data.head()

(41644, 4)


,criterion1,criterion2,criterion3,criterion4
45737,4.166667,3.750000,3.750000,4.166667
12903,0.833333,2.500000,1.250000,2.083333
55214,5.000000,4.166667,4.166667,5.000000
41336,1.250000,2.500000,1.666667,3.750000
57367,5.000000,5.000000,5.000000,5.000000


In [115]:
testing_data = full_test[['criterion1','criterion2','criterion3','criterion4']]
print(testing_data.shape)
testing_data.head()

(20512, 4)


,criterion1,criterion2,criterion3,criterion4
33907,5.000000,4.583333,3.750000,3.333333
25858,2.083333,2.500000,2.500000,2.500000
61694,5.000000,5.000000,5.000000,5.000000
1554,2.500000,1.666667,0.416667,0.416667
11072,3.333333,3.333333,3.750000,3.333333


In [116]:
predicted_dataset.columns = ['user_id','movie_id','criterion1','criterion2','criterion3','criterion4']
predicting_data = predicted_dataset[['criterion1','criterion2','criterion3','criterion4']]
print(predicting_data.shape)
predicting_data.head()

(20512, 4)


,criterion1,criterion2,criterion3,criterion4
0,4.169332,4.050744,3.828402,3.599499
1,2.716256,3.596563,2.796181,3.272879
2,3.602458,3.555491,2.993421,3.266680
3,3.661907,4.010367,4.188141,4.423180
4,3.897889,3.537554,3.450182,3.420344


In [117]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [118]:
lasso=Lasso()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)

In [119]:
lasso_regressor.fit(training_data,y_train)
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

{'alpha': 1e-15}
-0.2308463361817819


In [123]:
prediction_lasso=lasso_regressor.predict(predicting_data)

In [124]:
MSE_Lasso = np.square(np.subtract(y_test,prediction_lasso)).mean() 
RMSE_lasso = np.sqrt(np.mean((y_test-prediction_lasso)**2))
print(MSE_Lasso, ' is the MSE and ', RMSE_lasso, " is the RMSE for lasso regression")

2.8061817281830224  is the MSE and  1.6751661792738721  is the RMSE for lasso regression


In [125]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, prediction_lasso)

1.316062725481244